Кобзарь О.С. Горидько К.А. Хабибуллин Р.А.

## TODO сделать крутой аналитичейски dashboard с возможность настройки под различные исследования

# Работа с исходными данными исследований

Импорт модулей

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from _plotly_future_ import v4_subplots
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go

from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

from datetime import date

Загрузим файлик с данными из текущей директории и посмотрим на него. При необходимости нужно менять его название

In [ ]:
file = pd.read_csv("T18.trd", sep = ";", header = None)
file.head()

Произведем предварительную обработку и вновь посмотрим на файлик

In [ ]:
del file[0]
del file[1]
nedeed_columns = file.columns[2:]
for i in nedeed_columns:
    ost = i % 2
    if ost == 0 and i != 72:
        sensor_name = file[i][0]
    else:
        file = file.rename(columns={i:sensor_name})
        if i != 72:
            del file[i-1]
        
#del file['PLC_PRG.Diaf_Number']
file["Время"] = pd.to_datetime(file[2] + " " + file[3])
file.index = file["Время"]
del file[2]
del file[3]
del file["Время"]
file.head()


Проинициализируем функции для построения графиков

In [ ]:
def trace(data_x, data_y, namexy, chosen_mode = 'lines'):
    tracep = go.Scattergl(
        x=data_x,
        y=data_y,
        name=namexy,
        mode=chosen_mode
    )
    return tracep


def plot_func(plot_title_str, filename_str):
    layout = dict(title=plot_title_str)

    fig = dict(data=data, layout=layout)
    
    #fig = make_subplots(rows=8, cols=1)


    
    plot(fig, filename=filename_str)
    #iplot(fig)

def plot_subplots(data_traces, plot_title_str, filename_str, two_equal_subplots = False):
    if two_equal_subplots:
        items_in_one_subplot = int(len(data_traces) )
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
        for i in range(items_in_one_subplot):
            fig.append_trace(data_traces[i], row = 1, col = 1)
            fig.append_trace(data_traces[i], row = 2, col = 1)
    else:
        fig = make_subplots(rows=len(data_traces), cols=1, shared_xaxes=True, vertical_spacing=0.02)
        for i in range(len(data_traces)):
            fig.append_trace(data_traces[i], row = i + 1, col = 1)
    
    
    
    plot(fig, filename=filename_str)
    

def create_traces_list_by_num(data_x_values, data_y, num_y_list):
    trace_list = []
    for i in num_y_list:
        namexy = data_y.get_saved_parameter_name_by_number(i)
        this_trace = trace(data_x_values, data_y.get_saved_values_by_number(i), namexy)
        trace_list.append(this_trace)
    return trace_list

def create_traces_list_for_all_columms(data_frame, chosen_mode = 'lines'):
    trace_list = []
    columns_name_list = data_frame.columns
    for i in columns_name_list:
        column_name = i
        this_trace = trace(data_frame.index, data_frame[column_name], column_name, chosen_mode)
        trace_list.append(this_trace)
    return trace_list


def connect_traces(traces1, trace2):
    connected_traces = []
    for i in traces1:
        connected_traces.append(i)
    for j in trace2:
        connected_traces.append(j)
    return connected_traces

Построим графики для исходных данных

In [ ]:
data_traces = create_traces_list_for_all_columms(file)
data = data_traces
file_name = "Показания датчиков"
plot_func(file_name , file_name + ".html")

Определим интересующий промежуток времени для анализа

In [ ]:
year_start = 2019
month_start = 8
day_start = 15
hour_start = 19
minute_start = 1

year_end = 2019
month_end = 8
day_end = 15
hour_end = 20
minute_end = 3
date_from = pd.to_datetime(pd.DataFrame({'year': [year_start, year_end],
                           'month': [month_start, month_end],
                           'day': [day_start, day_end],
                            'hour': [hour_start, hour_end],
                                        'minute': [minute_start, minute_end]}))
date_start = date_from.iloc[0]
print("Дата начала периода для тщательного анализа: " + str(date_start))
date_end = date_from.iloc[1]
print("Дата конца периода для тщательного анализа: " + str(date_end))

Вырежем кусок исходных данным, посмотрим на него и продолжим работу только над ним

In [ ]:
nedeed_data = file[(file.index >= date_start) & (file.index <= date_end)]
nedeed_data.head()

Сделаем усреднение нужных данных и построим графики. Усреднение можно менять, например "1min", "2h"

In [ ]:
resample_time_str = "30s"
resampled_data = nedeed_data.resample(resample_time_str).mean()
data_traces = create_traces_list_for_all_columms(resampled_data)
data = data_traces
file_name = "Показания датчиков с осреднением " + resample_time_str
plot_func(file_name , file_name + ".html")


Построим первые 10 графиков

In [ ]:
traces_8_first = data_traces[0:8]
traces_2_last = data_traces[-3:]
traces_for_first_plot = connect_traces(traces_8_first, traces_2_last)
data = traces_for_first_plot
file_name = "Показания датчиков (первые 8 и тензо) с осреднением  " + resample_time_str
plot_func(file_name , file_name + ".html")

Еще вторые 8

In [ ]:
traces_8_second = data_traces[8:16]
data = traces_8_second
file_name = "Показания датчиков (вторые 8) с осреднением  " + resample_time_str
plot_func(file_name , file_name + ".html")

Еще третьи 8

In [ ]:
traces_8_third = data_traces[16:24]
data = traces_8_third
file_name = "Показания датчиков (третьи 8) с осреднением  " + resample_time_str
plot_func(file_name , file_name + ".html")

И последние 8

In [ ]:
traces_8_fourth = data_traces[16:24]
data = traces_8_fourth
file_name = "Показания датчиков (четвертые 8) с осреднением  " + resample_time_str
plot_func(file_name , file_name + ".html")

In [ ]:
ampl = nedeed_data[".Sensor1"] - nedeed_data[".Sensor1"].mean()
ampl